# MLP Reference Implementation in Pytorch.

This implementation is based on Pytorch's FashionMNIST example in its
[quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

See the mlax implementation in `mlp.ipynb` notebook.

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision

### Load the MNIST dataset.

In [2]:
mnist_train = torchvision.datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
mnist_test = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
print(mnist_train.data.shape)
print(mnist_test.data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


### Batch the MNIST data with Pytorch dataloaders.

In [3]:
# Note data is not shuffled
batch_size = 128
train_dataloader = DataLoader(mnist_train, batch_size, shuffle=True, num_workers=6)
test_dataloader = DataLoader(mnist_test, batch_size, num_workers=6)
print(len(train_dataloader), len(test_dataloader))

469 79


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


### Define MLP using modules.
We jit-compiled the model for better performance.

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, batch):
        flattened = self.flatten(batch)
        logits = self.linear_stack(flattened)
        return logits

model = MLP()
print(model)
model = torch.jit.script(model)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Define loss function and optimizer.
We also jit-compiled the loss function.

In [6]:
cross_entropy = torch.jit.script(nn.CrossEntropyLoss())
sgd = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

### Define training and testing loops.

In [7]:
def train(dataloader, model, loss_fn, optimizer, device):
    model.to(device)
    model.train()

    train_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        
        loss = loss_fn(model(X), y)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Train loss: {train_loss / len(dataloader)}")

In [8]:
def test(dataloader, model, loss_fn, device):
    model.to(device)
    model.eval()

    test_loss, accurate = 0.0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            accurate += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    print(f"Test loss: {test_loss / len(dataloader)}, accuracy: {accurate / len(dataloader.dataset)}")

In [9]:
def train_loop(
    train_dataloader,
    test_dataloader,
    model, loss_fn, optimizer,
    device,
    epochs, test_every):
    for i in range(epochs):
        epoch = (i + 1)
        print(f"Epoch {epoch}\n----------------")
        train(train_dataloader, model, loss_fn, optimizer, device)
        if (epoch % test_every == 0):
            test(test_dataloader, model, loss_fn, device)
        print(f"----------------")

### Train MLP on MNIST dataset.

In [10]:
train_loop(train_dataloader, test_dataloader, model, cross_entropy, sgd, device, 30, 5)

Epoch 1
----------------
Train loss: 0.7106316685676575
----------------
Epoch 2
----------------
Train loss: 0.26151955127716064
----------------
Epoch 3
----------------
Train loss: 0.1919376254081726
----------------
Epoch 4
----------------
Train loss: 0.14945566654205322
----------------
Epoch 5
----------------
Train loss: 0.12133917212486267
Test loss: 0.11110634011322562, accuracy: 0.9661
----------------
Epoch 6
----------------
Train loss: 0.0998171716928482
----------------
Epoch 7
----------------
Train loss: 0.0844765305519104
----------------
Epoch 8
----------------
Train loss: 0.07315052300691605
----------------
Epoch 9
----------------
Train loss: 0.06238320842385292
----------------
Epoch 10
----------------
Train loss: 0.054570648819208145
Test loss: 0.07346692386000783, accuracy: 0.9768
----------------
Epoch 11
----------------
Train loss: 0.04784969985485077
----------------
Epoch 12
----------------
Train loss: 0.04146675765514374
----------------
Epoch 13
-----